In [ ]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import math

In [ ]:
start_file  = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Long.mp4' #loading path of the original video

In [ ]:
dsv = deeplabcut.DownSampleVideo(start_file, width=320, height=240) # the video is being downsampled and its path is stored in variable dsv, with resolution of 320X240

In [ ]:
# Load the pre-trained face detection model using Open CV's dnn module, where deploy.prototxt.txt has the network architecture
#and res10_300x300_ssd_iter_140000_fp16.caffemodel contains the trained weights
model = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Open the video file
cap = cv2.VideoCapture(dsv) #capturing the video filoe for processing
fps = int(cap.get(cv2.CAP_PROP_FPS))#initialising variable for frames per rate. It extracts the frame rate of the video using cap.get

# Set the video writer, named CropGR, its codec, fps and frame size in pixels is being specified
fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec of the output video
out = cv2.VideoWriter('CropGr.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

check = False
frame_lum_val = []# initialising an empty list to store luminence value of each frame
target_br = 0.6 # setting the traget brightness(can range from 0 to 1)
target_g = math.log(target_br)# calculating the corresponding gamma value based on the brightness

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)# Setting the current frame position of the cv2.VideoCapture 
#object to the beginning of the video file (the first frame).

# Loop through the frames in the video
while True:
    # Read the next frame
    ret, frame = cap.read()#Read the next frame from the video and store it in the variable frame. 
    # boolean variable ret indicates whether the frame was successfully read.
    check = False
    
    # Check if the frame was successfully read
    if not ret:
        break
        
    # Resize the frame to a smaller size (for faster processing)
    small_frame = cv2.resize(frame, (300, 300))
    
    # Preprocess the frame for input to the neural network
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0)) ????
    
    # Pass the blob through the neural network to detect faces
    model.setInput(blob)
    detections = model.forward()
    #These lines pass the image blob through the pre-trained face detection model loaded in the previous line to detect faces in the frame. The setInput method sets the input blob to the network, 
    #and forward method performs a forward pass of the network on the input and returns the output detections.
    
     # Loop through the detections and draw rectangles around the monkey faces
    for i in range(detections.shape[2]): ???logic
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:  # Only show faces with high confidence+
            check = True
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            
            #identify region around face and perform adaptive gamma correction
            subframe = frame[startX:endX, startY:endY]
            subframe_normalized = subframe.astype(np.float32) / 255.0   #First, the subframe around the face is extracted from the frame and normalized to the range of 0 to 1.
            gray = cv2.cvtColor(subframe_normalized, cv2.COLOR_BGR2GRAY) #converted to gray scale to get brightness
            #The brightness of the subframe is computed by converting it to grayscale and taking the mean value.
            brightness = np.mean(gray)
            
            #finding and correcting gamma - a number is assigned to be the 'innate' gamma value of the frame, and that value is used to find the correction needed. 
            assigned_g = math.log(brightness)
            gamma = assigned_g/target_g
            # #This brightness value is used to compute the gamma correction factor by taking the natural logarithm of the brightness value
            #and dividing it by the target brightness value.
            
            #implementing gamma correction
            frame_normalized = frame.astype(np.float32) / 255.0
            
            frame_corrected = np.power(frame_normalized, 1/gamma)
            
            frame_scaled = (frame_corrected * 255.0).astype(np.uint8)
            
            # cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            
#In this part of the code, the detected faces from the previous part are used to perform gamma correction on the video frames.

#First, for each detected face, the confidence score is checked to ensure that it is above a certain threshold (0.7). If the confidence score is high enough, the region around the face is identified and normalized to the range of 0 to 1.

#Next, the average brightness of the region around the face is computed using grayscale conversion. The brightness value is then used to calculate a gamma correction factor that will be applied to the entire frame.

#The gamma correction is implemented by normalizing the frame to the range of 0 to 1, applying the gamma correction factor to each pixel value, and then scaling the pixel values back to the range of 0 to 255.

#Finally, the corrected frame is written to the output video file, and the loop continues until all frames have been processed. The loop can be interrupted by pressing the 'q' key.

            
    # cv2.imshow("Monkey faces", frame_scaled)
    if check:
        out.write(frame_scaled)
    
    # Wait for a key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video file and close the window
cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
os.mkdir("final_video") 
#create a new directory named final video

shutil.move('CropGr.mp4', 'final_video/CropGr.mp4')
#moves the output file to the final video directory

os.chdir('final_video')
#changes the current working directory to "final_video"

file = 'CropGr.mp4'
bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded! extracts the file extension and stores in video type
video_down = file# sets this variable to the name of the video file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face' #selects the model zoo's primate face model
project_name = 'DLC_GazeXBI'#name of the project
your_name = 'anc'#user name

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)

#creates a new project with DeepLabCut by calling the create_pretrained_project function, 
#and returns the configuration file paths and training configuration file paths as variables. 
#It used the previously mentioned model, project name, and user name. It also specifies 
#the input video file, the file type, and sets the "analyzevideo" and "copy_videos" flags to True. 
#It sets the "createlabeledvideo" flag to False to prevent the creation of a labeled video file.

NameError: name 'os' is not defined

In [ ]:
#create a dictionery 
edits = {
    'dotsize': 2,  # size of the dots on the output video!
    'pcutoff': 0.4,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
#auxiliary functions is called with the path to the project configuration file (config_path)
# and the dictionary of edits as arguments. It updates the project configuration file with the edits.

project_path = os.path.dirname(config_path) ??
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype, )
#This code calls the filterpredictions function from DeepLabCut with the project configuration file path (config_path), 
#the full path to the video file (full_video_path), and the video type (videotype) as arguments. This function filters the predictions made by the neural network, 
#removing any that are below the confidence threshold set in the project configuration file.

# re-create the video with your edits!
# deeplabcut.CropVideo(config_path, full_video_path, 'crop', 

deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
#Calling create_labeled_video function from DeepLabCut with the project configuration file path the full path to the video file , and the video type as arguments. 
#It also specifies the body parts to display in the output video , sets the draw_skeleton parameter to True to draw lines connecting the body parts, and sets the filtered parameter to True to use the filtered predictions. 
#This function generates a labeled video file that shows the predicted positions of the body parts in the original video.

In [ ]:
shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-28/videos/CropGrDLC_resnet50_DLC_GazeXBIMar28shuffle1_1030000_filtered_labeled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar28shuffle1_1030000_filtered_labeled.mp4')
#This line moves the labeled video file generated by DeepLabCut from a subdirectory (final_video/DLC_GazeXBI-anc-2023-03-28/videos/) #
#to the parent directory (/home/yramakrishna/DeepLabCut/conda-environments/).
try:
    shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video') #tries to remove final video directory
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))
try:
    os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Ken_Test_Longdownsampled.mp4')#tries to remove ken but why?
except OSError as e:
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))
    

In [ ]:
import tkinter as tk
from tkVideoPlayer import TkinterVideo

root = tk.Tk()

videoplayer = TkinterVideo(master=root, scaled=True)
videoplayer.load(r"/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar28shuffle1_1030000_filtered_labeled.mp4")
videoplayer.pack(expand=True, fill="both")

videoplayer.play() # play the video

root.mainloop()